<a href="https://colab.research.google.com/github/gurnoor6/opponent-modelling/blob/main/Opponent_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import shutil, os
# shutil.rmtree("game_logs")
shutil.unpack_archive('game_logs.zip', './')
shutil.unpack_archive('validation.zip', './')
print(len(os.listdir('game_logs')))
print(len(os.listdir('validation')))
train_filenames = os.listdir('game_logs')
val_filenames = os.listdir('validation')
train_filenames.sort()
val_filenames.sort()
print(train_filenames[0], val_filenames[0])

28005
19123
316756.npz 316756.npz


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Num GPUs Available:  1
Found GPU at: /device:GPU:0


In [ ]:
# Constants
T = 1 # Number of Timesteps for which boards are present in input
N = 8 # board size
HIDDEN_LAYER = 1 # hidden layer will be N * N * HIDDEN_LAYER neurons long
OUTPUT_LAYER = 73 # output layer will be N * N * OUTPUT_LAYER neurons long
BLOCKS = 10
FILTERS = 128

In [ ]:
import numpy as np

class My_Custom_Generator(tf.keras.utils.Sequence) :
  
  def __init__(self, filenames, batch_size, randomize) :
    self.filenames = filenames
    self.batch_size = batch_size
    self.randomize = randomize
    
    
  def __len__(self) :
    return (np.ceil(len(self.filenames) / float(self.batch_size))).astype(int)
  
  
  def __getitem__(self, idx) :
    filenames = self.filenames[idx * self.batch_size : (idx+1) * self.batch_size]

    inputs = []
    outputs = []
    for f in filenames:
      if not f.endswith('.npz'): continue
      loaded = np.load(os.path.join('game_logs',f))
      length = len(loaded['inputs'])
      index = np.random.choice(length, length,replace=False)
      if self.randomize:
        index = np.random.choice(length, min(3, length), replace = False)
      inputs.extend(loaded['inputs'][index])
      outputs.extend(loaded['outputs'][index])
    
    Y_new = []
    for item in outputs: Y_new.append(item.flatten())
    
    # print(np.array(inputs).shape, np.array(Y_new).shape)
    return np.array(inputs), np.array(Y_new)

In [ ]:
batch_size = 64

training_generator = My_Custom_Generator(train_filenames, batch_size, True)
validation_generator = My_Custom_Generator(val_filenames, batch_size, False)

In [ ]:
inputs = tf.keras.Input(shape=(8, 8, 14))
x1 = tf.keras.layers.LeakyReLU()(inputs)

x = tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x1)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.LeakyReLU()(x)

x = tf.keras.layers.Conv2D(14, (3, 3), activation="relu", padding="same")(x)
x = tf.keras.layers.BatchNormalization()(x)

added = tf.keras.layers.add([x1, x])
x = tf.keras.layers.LeakyReLU()(added)

x = tf.keras.layers.Conv2D(73, (3,3),activation="relu", padding="same")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.LeakyReLU()(x)

x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Softmax()(x)
model = tf.keras.Model(inputs, x)
model.summary()
# tf.keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 8, 8, 14)]   0           []                               
                                                                                                  
 leaky_re_lu_24 (LeakyReLU)     (None, 8, 8, 14)     0           ['input_7[0][0]']                
                                                                                                  
 conv2d_24 (Conv2D)             (None, 8, 8, 128)    16256       ['leaky_re_lu_24[0][0]']         
                                                                                                  
 batch_normalization_24 (BatchN  (None, 8, 8, 128)   512         ['conv2d_24[0][0]']              
 ormalization)                                                                              

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
)

In [ ]:
history = model.fit_generator(generator=training_generator,
                   steps_per_epoch = int(len(train_filenames) // batch_size),
                   epochs = 50,
                   verbose = 1,
                   validation_data = validation_generator,
                   validation_steps = int(1000 // batch_size)) # Use only 1000 samples for validation

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/50
437/437 [==============================] - 176s 401ms/step - loss: 4.2108 - categorical_accuracy: 0.2243 - val_loss: 6.2130 - val_categorical_accuracy: 0.1174
Epoch 2/50
437/437 [==============================] - 168s 384ms/step - loss: 3.2916 - categorical_accuracy: 0.2736 - val_loss: 3.9364 - val_categorical_accuracy: 0.1546
Epoch 3/50
437/437 [==============================] - 170s 388ms/step - loss: 3.1106 - categorical_accuracy: 0.2888 - val_loss: 3.7672 - val_categorical_accuracy: 0.1699
Epoch 4/50
437/437 [==============================] - 168s 384ms/step - loss: 3.0080 - categorical_accuracy: 0.2965 - val_loss: 3.6755 - val_categorical_accuracy: 0.1615
Epoch 5/50
437/437 [==============================] - 169s 387ms/step - loss: 2.9472 - categorical_accuracy: 0.3016 - val_loss: 3.5857 - val_categorical_accuracy: 0.1762
Epoch 6/50
437/437 [==============================] - 170s 390ms/step - loss: 2.9060 - categorical_accuracy: 0.3057 - val_loss: 3.5466 - val_categoric

KeyboardInterrupt: ignored

In [ ]:
# Plot the loss a sa function of number of epochs
import matplotlib.pyplot as plt
plt.title('Training Loss')
plt.xlabel('epochs')
plt.ylabel('Cross Entropy loss')
plt.plot(list(map(lambda x: x+1, range(len(history.history['loss'])))),  history.history['loss'])

In [ ]:
# Plot the loss a sa function of number of epochs
plt.title('Validation Loss')
plt.xlabel('epochs')
plt.ylabel('Cross Entropy loss')
plt.plot(list(map(lambda x: x+1, range(len(history.history['val_loss'])))),  history.history['val_loss'])

In [ ]:
print(history.history)